In [372]:
##Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1) 
pd.set_option("display.max_rows", 500)
sns.set(style='darkgrid')
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker


# import all libraries and dependencies for machine learning
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler

from sklearn.compose import make_column_transformer


import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy import stats
from scipy.stats import norm, kurtosis, skew

# Import specific libraries
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('expand_frame_repr', False)



In [373]:
##FILTRAR: mas_cols  = all_data.filter(like='Mas', axis=1).columns

In [374]:
#Loading updated dataset

%store -r all_data
%store -r y_train

In [375]:
all_data.shape

(2905, 242)

In [376]:
##TRANSFORM YEARS TO AGE: this solves the dilemma of whether years are ordinal or numerical...

#Busquemos todos los features relacionados con year
mas_cols  = all_data.filter(like='Year', axis=1).columns
mas_cols = mas_cols.append(all_data.filter(like='Yr', axis=1).columns)

In [377]:
mas_cols

Index(['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold'], dtype='object')

In [379]:
#Revisando se ve que 2010 es el año más reciente en todos los casos. Además hay valores zero
#Por tanto se convierte a age = 2011 - x si x no es cero y si lo es se deja en zero

for c in mas_cols:
    all_data[c] = all_data[c].apply(lambda x: 0 if x == 0 else 2011 - x)

In [385]:
##Mas features Deus Ex Machina
#[Remod] == indica binariamente si es o n
o remodelado
#["New"] == binario para ver si es nuevo
#[Age] == edad, entendida como la diferencia entyre remodelado y vendido

#Remod:
all_data["Remod"] = 100
all_data.loc[all_data["YearBuilt"] == all_data["YearRemodAdd"], ["Remod"]] = 0
all_data.loc[all_data["YearBuilt"] != all_data["YearRemodAdd"], ["Remod"]] = 1

#Remod:
all_data["New"] = 100
all_data.loc[all_data["YearBuilt"] == all_data["YearRemodAdd"], ["New"]] = 1
all_data.loc[all_data["YearBuilt"] != all_data["YearRemodAdd"], ["New"]] = 0

#Age
all_data["Age"] = all_data["YearRemodAdd"] - all_data["YearRemodAdd"]


